In [1]:
import pandas as pd


In [117]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv("C:\\sentrymind\\backend\\data\\fraudTrain.csv")

# Convert timestamp to datetime
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])

# Sort transactions by customer and time
df = df.sort_values(by=['cc_num', 'trans_date_trans_time'])

# Verify first few rows
df.head()


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
1017,1017,2019-01-01 12:47:15,60416207185,"fraud_Jones, Sawayn and Romaguera",misc_net,7.27,Mary,Diaz,F,9886 Anita Drive,...,43.0048,-108.8964,1645,Information systems manager,1986-02-17,98e3dcf98101146a577f85a34e58feec,1325422035,43.974711,-109.741904,0
2724,2724,2019-01-02 08:44:57,60416207185,fraud_Berge LLC,gas_transport,52.94,Mary,Diaz,F,9886 Anita Drive,...,43.0048,-108.8964,1645,Information systems manager,1986-02-17,498120fc45d277f7c88e3dba79c33865,1325493897,42.018766,-109.044172,0
2726,2726,2019-01-02 08:47:36,60416207185,fraud_Luettgen PLC,gas_transport,82.08,Mary,Diaz,F,9886 Anita Drive,...,43.0048,-108.8964,1645,Information systems manager,1986-02-17,95f514bb993151347c7acdf8505c3d62,1325494056,42.961335,-109.157564,0
2882,2882,2019-01-02 12:38:14,60416207185,fraud_Daugherty LLC,kids_pets,34.79,Mary,Diaz,F,9886 Anita Drive,...,43.0048,-108.8964,1645,Information systems manager,1986-02-17,4f0c1a14e0aa7eb56a490780ef9268c5,1325507894,42.228227,-108.747683,0
2907,2907,2019-01-02 13:10:46,60416207185,fraud_Beier and Sons,home,27.18,Mary,Diaz,F,9886 Anita Drive,...,43.0048,-108.8964,1645,Information systems manager,1986-02-17,3b2ebd3af508afba959640893e1e82bc,1325509846,43.321745,-108.091143,0


In [119]:
df['transaction_amount'] = df['amt']


In [121]:
# Assume merchant risk scores are precomputed and stored in a dictionary
merchant_risk_scores = {
    'merchant_1': 7.5, 'merchant_2': 2.3, 'merchant_3': 9.0,  # Example values
}
df['merchant_risk_score'] = df['merchant'].map(merchant_risk_scores)


In [125]:
# Convert timestamp column to datetime
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])

# Sort transactions chronologically within each customer group
df = df.sort_values(by=['cc_num', 'trans_date_trans_time'])

# ✅ Corrected approach: Use `.apply()` within `groupby`
def count_past_24h_transactions(group):
    return group.rolling('1d', on='trans_date_trans_time').count()

df['transaction_count_24h'] = df.groupby('cc_num', group_keys=False).apply(count_past_24h_transactions)['trans_num']


C:\Users\gandh\AppData\Local\Temp\ipykernel_32428\3186324961.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['transaction_count_24h'] = df.groupby('cc_num', group_keys=False).apply(count_past_24h_transactions)['trans_num']


In [133]:
# Convert timestamp column to datetime
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])

# Sort transactions chronologically within each customer
df = df.sort_values(by=['cc_num', 'trans_date_trans_time'])

# Define function to compute 30-day average spend
def calculate_avg_spend(group):
    group = group.set_index('trans_date_trans_time').sort_index()  # Set index for rolling window
    group['avg_spend_last_30d'] = group['amt'].rolling('30D').mean()  # Apply rolling mean
    return group.reset_index()  # Reset index back to original format

# Apply function while maintaining DataFrame structure
df = df.groupby('cc_num', group_keys=False).apply(calculate_avg_spend)


C:\Users\gandh\AppData\Local\Temp\ipykernel_32428\2551260755.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('cc_num', group_keys=False).apply(calculate_avg_spend)


In [135]:
def max_spend_last_30d(group):
    group = group.set_index('trans_date_trans_time').sort_index()
    group['max_spend_last_30d'] = group['amt'].rolling('30D').max()
    return group.reset_index()

df = df.groupby('cc_num', group_keys=False).apply(max_spend_last_30d)


C:\Users\gandh\AppData\Local\Temp\ipykernel_32428\362746785.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('cc_num', group_keys=False).apply(max_spend_last_30d)


In [137]:
def min_spend_last_30d(group):
    group = group.set_index('trans_date_trans_time').sort_index()
    group['min_spend_last_30d'] = group['amt'].rolling('30D').min()
    return group.reset_index()

df = df.groupby('cc_num', group_keys=False).apply(min_spend_last_30d)


C:\Users\gandh\AppData\Local\Temp\ipykernel_32428\1502302882.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('cc_num', group_keys=False).apply(min_spend_last_30d)


In [147]:
!pip install geopy

   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/125.4 kB ? eta -:--:--
   ------------ -------------------------- 41.0/125.4 kB 487.6 kB/s eta 0:00:01
   ---------------------------------------- 125.4/125.4 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
   ---------------------------------------- 40.3/40.3 kB ? eta 0:00:00


In [163]:
# Import required libraries
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import warnings
from datetime import datetime, timedelta

# Suppress warnings
warnings.filterwarnings('ignore')

# Load dataset
print("Loading dataset...")
df = pd.read_csv("C:\\sentrymind\\backend\\data\\fraudTrain.csv")

# Convert timestamps to datetime
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
df['dob'] = pd.to_datetime(df['dob'])

# Sort transactions chronologically within each customer
df = df.sort_values(by=['cc_num', 'trans_date_trans_time'])

print("Computing features...")

# ✅ Compute Transaction Count in Last 24 Hours
def count_past_24h_transactions(group):
    try:
        group = group.copy()
        group = group.set_index('trans_date_trans_time')
        group['transaction_count_24h'] = group.rolling('24H', min_periods=1)['trans_num'].count()
        return group.reset_index()
    except Exception as e:
        print(f"Error in 24h transactions: {e}")
        group['transaction_count_24h'] = 0
        return group

df = df.groupby('cc_num', group_keys=False).apply(count_past_24h_transactions)
print("✓ 24-hour transaction counts computed")

# ✅ Compute Rolling Statistics (30-day window)
def calculate_rolling_stats(group):
    try:
        group = group.copy()
        group = group.set_index('trans_date_trans_time')
        
        # Calculate all rolling statistics
        rolling_window = group['amt'].rolling('30D', min_periods=1)
        group['avg_spend_last_30d'] = rolling_window.mean()
        group['max_spend_last_30d'] = rolling_window.max()
        group['min_spend_last_30d'] = rolling_window.min()
        group['monthly_spend_avg'] = group['avg_spend_last_30d'] * 30
        
        return group.reset_index()
    except Exception as e:
        print(f"Error in rolling stats: {e}")
        for col in ['avg_spend_last_30d', 'max_spend_last_30d', 'min_spend_last_30d', 'monthly_spend_avg']:
            group[col] = np.nan
        return group

df = df.groupby('cc_num', group_keys=False).apply(calculate_rolling_stats)
print("✓ Rolling statistics computed")

# ✅ Compute Number of Unique Merchants in Last 30 Days
# ✅ Compute Number of Unique Merchants in Last 30 Days - Optimized Version
def unique_merchants_last_30d(group):
    try:
        group = group.copy()
        group = group.set_index('trans_date_trans_time')
        
        # Create a helper function to count unique merchants efficiently
        def count_unique_in_window(window_data):
            if len(window_data) == 0:
                return 0
            return len(set(window_data))
        
        # Use rolling apply with the optimized function
        group['unique_merchants_30d'] = (
            group['merchant']
            .rolling('30D', min_periods=1)
            .agg(count_unique_in_window)
        )
        
        return group.reset_index()
    except Exception as e:
        print(f"Error in unique merchants: {e}")
        group['unique_merchants_30d'] = 0
        return group

# ✅ Compute Number of Past Fraud Transactions in Last 30 Days
def past_fraud_last_30d(group):
    try:
        group = group.copy()
        group = group.set_index('trans_date_trans_time')
        group['past_fraud_30d'] = group['is_fraud'].rolling('30D', min_periods=1).sum()
        return group.reset_index()
    except Exception as e:
        print(f"Error in past fraud: {e}")
        group['past_fraud_30d'] = 0
        return group

df = df.groupby('cc_num', group_keys=False).apply(past_fraud_last_30d)
print("✓ Past fraud counts computed")

# ✅ Compute Account Age in Days
df['account_age_days'] = (df['trans_date_trans_time'] - df['dob']).dt.days
print("✓ Account age computed")

# ✅ Compute Distance Between Merchant and Customer
# ✅ Compute Distance Between Merchant and Customer - Optimized Version
print("Computing distances (this may take a few minutes)...")

# Vectorize the distance calculation for better performance
def compute_distances_vectorized(df):
    try:
        # Create masks for valid coordinates
        valid_coords = (
            df['lat'].notna() & 
            df['long'].notna() & 
            df['merch_lat'].notna() & 
            df['merch_long'].notna()
        )
        
        # Initialize distances array with NaN
        distances = np.full(len(df), np.nan)
        
        # Only compute for valid coordinates
        valid_df = df[valid_coords]
        
        # Process in chunks to avoid memory issues
        chunk_size = 10000
        for i in range(0, len(valid_df), chunk_size):
            chunk = valid_df.iloc[i:i+chunk_size]
            chunk_distances = [
                geodesic(
                    (row['lat'], row['long']),
                    (row['merch_lat'], row['merch_long'])
                ).miles
                for _, row in chunk.iterrows()
            ]
            distances[valid_df.index[i:i+chunk_size]] = chunk_distances
            
            # Print progress
            if i % 50000 == 0:
                print(f"Processed {i}/{len(valid_df)} records...")
        
        return distances
        
    except Exception as e:
        print(f"Error in distance computation: {e}")
        return np.full(len(df), np.nan)

# Compute distances
df['customer_merchant_distance'] = compute_distances_vectorized(df)
print("✓ Customer-merchant distances computed")



Loading dataset...
Computing features...
✓ 24-hour transaction counts computed
✓ Rolling statistics computed
✓ Past fraud counts computed
✓ Account age computed
Computing distances (this may take a few minutes)...
Processed 0/1296675 records...
Processed 50000/1296675 records...
Processed 100000/1296675 records...
Processed 150000/1296675 records...
Processed 200000/1296675 records...
Processed 250000/1296675 records...
Processed 300000/1296675 records...
Processed 350000/1296675 records...
Processed 400000/1296675 records...
Processed 450000/1296675 records...
Processed 500000/1296675 records...
Processed 550000/1296675 records...
Processed 600000/1296675 records...
Processed 650000/1296675 records...
Processed 700000/1296675 records...
Processed 750000/1296675 records...
Processed 800000/1296675 records...
Processed 850000/1296675 records...
Processed 900000/1296675 records...
Processed 950000/1296675 records...
Processed 1000000/1296675 records...
Processed 1050000/1296675 records..

KeyError: 'merchant_state'

In [165]:
print("Available columns in dataset:")
print(df.columns.tolist())

Available columns in dataset:
['trans_date_trans_time', 'Unnamed: 0', 'cc_num', 'merchant', 'category', 'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud', 'transaction_count_24h', 'avg_spend_last_30d', 'max_spend_last_30d', 'min_spend_last_30d', 'monthly_spend_avg', 'past_fraud_30d', 'account_age_days', 'customer_merchant_distance']


In [175]:
def unique_merchants_last_30d_ultrafast(group):
    try:
        # Downsample to 3-day periods for extreme speed
        group = group.copy()
        group['date'] = group['trans_date_trans_time'].dt.date
        three_day_merchants = (
            group.groupby(group['date'].map(lambda x: x - pd.Timedelta(days=x.day % 3)))
            ['merchant'].agg(set)
        )
        group['unique_merchants_30d'] = group['date'].map(
            lambda x: len(set().union(*three_day_merchants[
                (three_day_merchants.index <= x) & 
                (three_day_merchants.index > x - pd.Timedelta(days=30))
            ]))
        )
        return group
    except Exception as e:
        print(f"Error: {e}")
        group['unique_merchants_30d'] = 0
        return group

In [179]:
# ✅ Compute Number of Unique Merchants in Last 30 Days - Ultra Fast Version
def unique_merchants_last_30d_ultrafast(group):
    try:
        # Downsample to 3-day periods for extreme speed
        group = group.copy()
        group['date'] = group['trans_date_trans_time'].dt.date
        three_day_merchants = (
            group.groupby(group['date'].map(lambda x: x - pd.Timedelta(days=x.day % 3)))
            ['merchant'].agg(set)
        )
        group['unique_merchants_30d'] = group['date'].map(
            lambda x: len(set().union(*three_day_merchants[
                (three_day_merchants.index <= x) & 
                (three_day_merchants.index > x - pd.Timedelta(days=30))
            ]))
        )
        return group
    except Exception as e:
        print(f"Error: {e}")
        group['unique_merchants_30d'] = 0
        return group

print("Computing unique merchants (this should take 5-7 minutes)...")
df = df.groupby('cc_num', group_keys=False).apply(unique_merchants_last_30d_ultrafast)
print("✓ Unique merchants computed")

# Now let's verify all features again
expected_features = [
    'transaction_count_24h', 
    'avg_spend_last_30d', 
    'max_spend_last_30d',
    'min_spend_last_30d', 
    'unique_merchants_30d',
    'past_fraud_30d',
    'account_age_days', 
    'monthly_spend_avg', 
    'customer_merchant_distance'
]

print("\nVerifying all features:")
for feature in expected_features:
    exists = feature in df.columns
    print(f"{feature}: {'✓' if exists else '✗'}")

# If successful, save the updated dataset
if 'unique_merchants_30d' in df.columns:
    df.to_csv("fraud_features_engineered.csv", index=False)
    print("\nDataset saved with all features including unique_merchants_30d")

Computing unique merchants (this should take 5-7 minutes)...
✓ Unique merchants computed

Verifying all features:
transaction_count_24h: ✓
avg_spend_last_30d: ✓
max_spend_last_30d: ✓
min_spend_last_30d: ✓
unique_merchants_30d: ✓
past_fraud_30d: ✓
account_age_days: ✓
monthly_spend_avg: ✓
customer_merchant_distance: ✓

Dataset saved with all features including unique_merchants_30d


In [183]:
# Load and examine the engineered dataset
import pandas as pd
import numpy as np

# Load the data
print("Loading engineered dataset...")
df = pd.read_csv("fraud_features_engineered.csv")

# 1. Check all columns
print("\n1. Available columns:")
print(df.columns.tolist())

# 2. Check data completeness
print("\n2. Missing values in each column:")
print(df.isnull().sum())

# 3. Check target variable distribution
print("\n3. Fraud distribution:")
print(df['is_fraud'].value_counts(normalize=True))

# 4. Check engineered features statistics
engineered_features = [
    'transaction_count_24h', 
    'avg_spend_last_30d', 
    'max_spend_last_30d',
    'min_spend_last_30d', 
    'unique_merchants_30d',
    'past_fraud_30d',
    'account_age_days', 
    'monthly_spend_avg', 
    'customer_merchant_distance'
]

print("\n4. Engineered features statistics:")
print(df[engineered_features].describe())

# 5. Check categorical features
categorical_features = ['merchant', 'category']
print("\n5. Number of unique values in categorical features:")
for col in categorical_features:
    if col in df.columns:
        print(f"{col}: {df[col].nunique()} unique values")

# 6. Check data types
print("\n6. Data types of columns:")
print(df.dtypes)

# 7. Check for any anomalies in key features
print("\n7. Checking for anomalies in key features:")
for feature in engineered_features:
    if feature in df.columns:
        print(f"\n{feature}:")
        print("Min:", df[feature].min())
        print("Max:", df[feature].max())
        print("% zeros:", (df[feature] == 0).mean() * 100)

Loading engineered dataset...

1. Available columns:
['trans_date_trans_time', 'Unnamed: 0', 'cc_num', 'merchant', 'category', 'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud', 'transaction_count_24h', 'avg_spend_last_30d', 'max_spend_last_30d', 'min_spend_last_30d', 'monthly_spend_avg', 'past_fraud_30d', 'account_age_days', 'customer_merchant_distance', 'date', 'unique_merchants_30d']

2. Missing values in each column:
trans_date_trans_time               0
Unnamed: 0                          0
cc_num                              0
merchant                            0
category                            0
amt                                 0
first                               0
last                                0
gender                              0
street                              0
city                                0
state                             

In [185]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import pandas as pd
import numpy as np

# 1. Prepare features
features_to_use = [
    # Amount-related
    'amt', 'avg_spend_last_30d', 'max_spend_last_30d', 'min_spend_last_30d', 'monthly_spend_avg',
    
    # Transaction patterns
    'transaction_count_24h', 'unique_merchants_30d', 'past_fraud_30d',
    
    # Location/Time
    'customer_merchant_distance', 'account_age_days',
    
    # Categorical
    'merchant', 'category'
]

# 2. Encode categorical variables
le_merchant = LabelEncoder()
le_category = LabelEncoder()

df['merchant_encoded'] = le_merchant.fit_transform(df['merchant'])
df['category_encoded'] = le_category.fit_transform(df['category'])

# Update features to use encoded versions
features_to_use = [f if f not in ['merchant', 'category'] else f+'_encoded' for f in features_to_use]

# 3. Split the data
X = df[features_to_use].fillna(-999)  # Simple handling of missing values
y = df['is_fraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 4. Setup XGBoost with parameters suited for imbalanced data
xgb_params = {
    'max_depth': 6,
    'min_child_weight': 1,
    'eta': 0.3,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective': 'binary:logistic',
    'scale_pos_weight': len(y_train[y_train==0]) / len(y_train[y_train==1]),  # Handle class imbalance
    'eval_metric': 'auc',
    'tree_method': 'hist'  # For faster training
}

# 5. Train model
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Train with early stopping
model = xgb.train(
    xgb_params,
    dtrain,
    num_boost_round=1000,
    evals=[(dtrain, 'train'), (dtest, 'test')],
    early_stopping_rounds=50,
    verbose_eval=50
)

# 6. Make predictions
y_pred_proba = model.predict(dtest)
y_pred = (y_pred_proba > 0.5).astype(int)

# 7. Print model performance
print("\nModel Performance:")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nROC AUC Score:", roc_auc_score(y_test, y_pred_proba))

# 8. Feature importance analysis
importance = model.get_score(importance_type='gain')
sorted_importance = sorted(importance.items(), key=lambda x: x[1], reverse=True)
print("\nTop 10 Most Important Features:")
for feature, score in sorted_importance[:10]:
    print(f"{feature}: {score}")

ModuleNotFoundError: No module named 'xgboost'

In [187]:
!pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB 330.3 kB/s eta 0:06:19
   ---------------------------------------- 0.1/124.9 MB 751.6 kB/s eta 0:02:47
   ---------------------------------------- 0.4/124.9 MB 2.8 MB/s eta 0:00:46
   ---------------------------------------- 1.2/124.9 MB 6.1 MB/s eta 0:00:21
   - -------------------------------------- 4.5/124.9 MB 17.9 MB/s eta 0:00:07
   - -------------------------------------- 4.8/124.9 MB 17.0 MB/s eta 0:00:08
   -- ------------------------------------- 7.2/124.9 MB 21.0 MB/s eta 0:00:06
   -- ------------------------------------- 8.8/124.9 MB 22.6 MB/s eta 0:00:06
   ---- ----------------------------------- 13.0/124.9 MB 50.4 MB/s eta 0:00:03
   ----- ---------------------------------- 17.3/124.9 MB 72.6 MB/s eta 0:00:02
   ------- -------------------------------- 22.1/124.9 MB 93.9 MB/

In [189]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import pandas as pd
import numpy as np

# 1. Prepare features
features_to_use = [
    # Amount-related
    'amt', 'avg_spend_last_30d', 'max_spend_last_30d', 'min_spend_last_30d', 'monthly_spend_avg',
    
    # Transaction patterns
    'transaction_count_24h', 'unique_merchants_30d', 'past_fraud_30d',
    
    # Location/Time
    'customer_merchant_distance', 'account_age_days',
    
    # Categorical
    'merchant', 'category'
]

# 2. Encode categorical variables
le_merchant = LabelEncoder()
le_category = LabelEncoder()

df['merchant_encoded'] = le_merchant.fit_transform(df['merchant'])
df['category_encoded'] = le_category.fit_transform(df['category'])

# Update features to use encoded versions
features_to_use = [f if f not in ['merchant', 'category'] else f+'_encoded' for f in features_to_use]

# 3. Split the data
X = df[features_to_use].fillna(-999)  # Simple handling of missing values
y = df['is_fraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 4. Setup XGBoost with parameters suited for imbalanced data
xgb_params = {
    'max_depth': 6,
    'min_child_weight': 1,
    'eta': 0.3,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective': 'binary:logistic',
    'scale_pos_weight': len(y_train[y_train==0]) / len(y_train[y_train==1]),  # Handle class imbalance
    'eval_metric': 'auc',
    'tree_method': 'hist'  # For faster training
}

# 5. Train model
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Train with early stopping
model = xgb.train(
    xgb_params,
    dtrain,
    num_boost_round=1000,
    evals=[(dtrain, 'train'), (dtest, 'test')],
    early_stopping_rounds=50,
    verbose_eval=50
)

# 6. Make predictions
y_pred_proba = model.predict(dtest)
y_pred = (y_pred_proba > 0.5).astype(int)

# 7. Print model performance
print("\nModel Performance:")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nROC AUC Score:", roc_auc_score(y_test, y_pred_proba))

# 8. Feature importance analysis
importance = model.get_score(importance_type='gain')
sorted_importance = sorted(importance.items(), key=lambda x: x[1], reverse=True)
print("\nTop 10 Most Important Features:")
for feature, score in sorted_importance[:10]:
    print(f"{feature}: {score}")

[0]	train-auc:0.99525	test-auc:0.99517
[50]	train-auc:0.99998	test-auc:0.99993
[100]	train-auc:1.00000	test-auc:0.99995
[150]	train-auc:1.00000	test-auc:0.99995
[200]	train-auc:1.00000	test-auc:0.99996
[250]	train-auc:1.00000	test-auc:0.99996
[300]	train-auc:1.00000	test-auc:0.99996
[350]	train-auc:1.00000	test-auc:0.99996
[372]	train-auc:1.00000	test-auc:0.99996

Model Performance:

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    257834
           1       0.94      0.97      0.96      1501

    accuracy                           1.00    259335
   macro avg       0.97      0.99      0.98    259335
weighted avg       1.00      1.00      1.00    259335


Confusion Matrix:
[[257740     94]
 [    40   1461]]

ROC AUC Score: 0.9999585151588555

Top 10 Most Important Features:
past_fraud_30d: 3810.491455078125
max_spend_last_30d: 209.3304443359375
category_encoded: 140.88954162597656
amt: 127.01374816894531
transact

In [193]:
# In your Jupyter notebook, first let's see current directory
import os
print("Current working directory:", os.getcwd())

# Create a models directory if it doesn't exist
model_dir = "models"
os.makedirs(model_dir, exist_ok=True)

# Now save the model and encoders in the models directory
model_path = os.path.join(model_dir, "fraud_detection_model.json")
merchant_encoder_path = os.path.join(model_dir, "fraud_detection_merchant_encoder.joblib")
category_encoder_path = os.path.join(model_dir, "fraud_detection_category_encoder.joblib")

# Save the model and encoders
model.save_model(model_path)
joblib.dump(le_merchant, merchant_encoder_path)
joblib.dump(le_category, category_encoder_path)

print("\nModel saved at:", model_path)
print("Merchant encoder saved at:", merchant_encoder_path)
print("Category encoder saved at:", category_encoder_path)

Current working directory: C:\Users\gandh

Model saved at: models\fraud_detection_model.json
Merchant encoder saved at: models\fraud_detection_merchant_encoder.joblib
Category encoder saved at: models\fraud_detection_category_encoder.joblib
